# Main Commands

## Fetch Images

### Download card images from Scryfall queries

In [ ]:
from art_scan import process_query

queries = [
    "Crucible of Worlds",
]

[process_query(q) for q in queries]
# [process_query(q, skip_scryfall=True) for q in queries]
# [process_query(q, skip_mtgpics=True) for q in queries]

## Fetch Set Symbols

### Parse Keyrune website cheatsheet and print out UTF8 representation

In [ ]:
from bs4 import BeautifulSoup
import requests

url = "https://keyrune.andrewgioia.com/cheatsheet.html"

response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")
set_symbol_spans = soup.findAll("span", {"class": "utf"})
print(f"Found {len(set_symbol_spans)} icons.")

In [ ]:
set_symbols = {}
for set_symbol_span in set_symbol_spans:
    set_symbol = str(set_symbol_span.findChild("i").get_text())
    try:
        set_code = set_symbol_span.get_text().split("ss-")[1].split(" ")[0].upper()
    except IndexError as e:
        continue
    finally:
        set_symbols[set_code] = set_symbol

for set_code, set_symbol in set_symbols.items():
    print(f"\"{set_code}\": \"{set_symbol}\",")

## Inventory Management

### Check for image files

In [ ]:
from lib import os_helpers

ART_FOLDER = "C:/Users/evanh/My Drive/MTG/Proxies/art"

to_find = [
    "Crucible of Worlds"
]

os_helpers.find_missing_files(ART_FOLDER, to_find)

### Generate inventory as TSV

In [ ]:
import operator
from lib import os_helpers, scryfall_helpers
from lib.classes import InventoryCard

PROXY_FOLDER = "C:/Users/evanh/My Drive/MTG/Proxies/mpc-ready"
IGNORED_FILES = ["desktop.ini", "inventory.tsv", "Custom", "MDFC"]
INVENTORY_FILE_NAME = "inventory.tsv"
MDFC_FOLDER_NAME = "MDFC"

# Process all normal cards first
inventory_cards = []
for card_type in os_helpers.list_files(PROXY_FOLDER, ignore=IGNORED_FILES):
    typed_proxy_folder = f"{PROXY_FOLDER}/{card_type}"
    for file_name in os_helpers.list_files(typed_proxy_folder, ignore=IGNORED_FILES):
        card = InventoryCard(
            name=".".join(file_name.split(".")[:-1]),  # Strip off file extension
            modified=os_helpers.get_modified_date_utc(f"{typed_proxy_folder}/{file_name}"),
            type_=card_type
        )
        inventory_cards.append(card)

# Process MDFC card images as a single entry
mdfc_proxy_folder = f"{PROXY_FOLDER}/{MDFC_FOLDER_NAME}"
unique_mdfcs = dict()
for file_name in os_helpers.list_files(mdfc_proxy_folder, ignore=IGNORED_FILES):
    name = ".".join(file_name.split(".")[:-1])  # Strip off file extension
    modified = os_helpers.get_modified_date_utc(f"{mdfc_proxy_folder}/{file_name}")
    card = scryfall_helpers.get_named_card(name)
    
    unique_mdfcs[card.name] = dict(modified=modified, type_=card.folder_name)

# Generate list of unique MDFCs
for card_name, details in unique_mdfcs.items():
    card = InventoryCard(name=card_name, modified=details["modified"], type_=details["type_"])
    inventory_cards.append(card)

# Sort inventory to match destination spreadsheet
sorted_cards = sorted(inventory_cards, key=operator.itemgetter("type_", "name"))

# Write list of cards to TSV
inventory_file_path = f"{PROXY_FOLDER}/{INVENTORY_FILE_NAME}"
with open(inventory_file_path, "w") as f:
    for card in sorted_cards:
        f.write(card.tsv + "\n")
    print(f"Created inventory at: \"{inventory_file_path}\"")


## Order Generation

### Pull proxy list from Google Sheets (public)

In [ ]:
from lib.classes import Proxy
from lib import common

sheet_id = '1JbJy3kM34XUOPud4qPaAhp2luU0Qgidq4CPQVm0TTw0'
url = f'https://docs.google.com/spreadsheet/ccc?key={sheet_id}&output=csv'

proxy_csv_data = common.load_csv(url.format(sheet_id))

all_proxies = [Proxy.from_row(p) for p in proxy_csv_data[4:] if len(p[0]) > 0]
proxies_to_print = [p for p in all_proxies if p.order_count > 0]
len(proxies_to_print)

### Pull proxy list from Google Sheets (private)

In [ ]:
from googleapiclient import discovery
from lib.classes import Proxy
from lib.google_auth import get_creds

args = dict(
    spreadsheetId='13ofTLZFmPMkT09Np2US8KF_JmTfgqdeQ5aeY16L43s0',
    range='Inventory!A:L',
)

service = discovery.build('sheets', 'v4', credentials=get_creds())
result = service.spreadsheets().values().get(**args).execute()
proxy_csv_data = list(result.get('values', []))

all_proxies = [Proxy.from_row(p) for p in proxy_csv_data[4:] if len(p[0]) > 0]
proxies_to_print = [p for p in all_proxies if p.order_count > 0]
len(proxies_to_print)

### Find missing proxy files

In [ ]:
from lib import os_helpers

proxy_folder = "C:/Users/evanh/My Drive/MTG/Proxies/mpc-ready"

card_types = list(set([p.type for p in proxies_to_print]))
proxies_by_type = {
    type_: [p for p in proxies_to_print if p.type == type_]
    for type_ in card_types
}

missing_files = []
for card_type in card_types:
    folder_name = f"{proxy_folder}/{card_type}"
    proxy_file_names = [p.name for p in proxies_by_type[card_type]]
    missing_files += os_helpers.find_missing_files(folder_name, proxy_file_names)

print("\n".join(missing_files))

### Write out files for order

In [ ]:
proxies = [p for p in proxies_to_print if not p.is_mdfc]
# proxies = [p for p in proxies_to_print if p.is_mdfc]
len(proxies)

In [ ]:
from lib import os_helpers, proxy_helpers
import importlib

importlib.reload(os_helpers)
importlib.reload(proxy_helpers)

folder_size = 612
prefix_name = "proxies_{number:02d}"
landing_folder = "C:/Users/evanh/Temp/landing"
output_folder = "C:/Users/evanh/Temp/mpc"
proxy_folder = "C:/Users/evanh/My Drive/MTG/Proxies/mpc-ready"

# Write all files out to landing zone
missing, duplicates = proxy_helpers.create_unique_proxies(proxies, proxy_folder, landing_folder)
if len(missing):
    print("Missing:\n\t{}".format('\n\t'.join([p.name for p in missing])))
if len(duplicates):
    print("Duplicates:\n\t{}".format('\n\t'.join([p.name for p in duplicates])))

# Move files into subfolders of fixed size (`folder_size`)
output_folders = os_helpers.split_files(landing_folder, output_folder, folder_size, prefix_name)
output_folders

# Debug Commands

## Save card info to `card.json`

In [ ]:
from scripts import get_card_info

args = "\"Angelic Accord\" \"M14\""
get_card_info.main(args)

## Scryfall

### Get multiple cards

In [ ]:
from lib.scryfall_helpers import get_matched_cards

queries = [
    "Crucible of Worlds",
]

for q in queries:
    cards = get_matched_cards(q)
    [print(f"\t{repr(c)}") for c in cards]

### Get single card

In [ ]:
from lib.scryfall_helpers import get_named_card

queries = [
    "Crucible of Worlds set=5dn",
]

for q in queries:
    card = get_named_card(q)
    print(repr(card))

## MTGPICS.com

### Find `gamerid`

In [ ]:
from lib.mtgpics_helpers import get_gamerid

queries = [
    "Crucible of Worlds",
]

for q in queries:
    card_name, gamerid = get_gamerid(q)
    print(f"Found id [{gamerid}] for [{card_name}]")

### Find image versions by `gamerid`

In [ ]:
from lib.mtgpics_helpers import get_gamerid, find_all_art_versions

to_find = "Crucible of Worlds"
card_name, gamerid = to_find, get_gamerid(to_find)
versions = find_all_art_versions(card_name, gamerid)
versions

## Misc